In [22]:
# Library import
import pandas as pd
import numpy as np
import re
import pymysql

from sqlalchemy import create_engine
from pandas.util import hash_pandas_object

In [23]:
# Read data
orders = pd.read_csv("./data/orders_ccv.csv")

In [24]:
# Only include columns we need
orders = orders[orders.columns[orders.columns.isin(["Ordernummer","Postcode","Woonplaats","Land","Datum","E-mail","Prijs excl BTW","Verzendkosten","Aantal","ProductNaam"])]]

# Rename columns
orders.columns = ["order_number","date","item_price","shipping","postal_code","city","country","email","item_name","quantity"]


In [25]:
# Change dtypes 
orders["item_price"] = orders["item_price"].str.replace(",",".")
orders["item_price"] = orders["item_price"].astype(float)

orders["shipping"] = orders["shipping"].astype(str)
orders["shipping"] = orders["shipping"].str.replace(",",".")
orders["shipping"] = orders["shipping"].astype(float)

orders["email"] = orders["email"].str.lower()

# Calculate order_total
orders["order_total"] = orders["item_price"]-orders["shipping"]

In [26]:
# Where product is a yerba mate set, with child items, set quantity to zero
qty2 = (orders.item_name == "Yerba Mate Set 100 gram incl. kalebas")
qty3 = (orders.item_name == "Yerba mate set 100 gram incl. kalebas en bombilla") | (
    orders.item_name == "Yerba Mate Set 250 gram")|(
    orders.item_name == "Yerba Mate Set 100 gram")|(
    orders.item_name == "Yerba Mate Set 300 gram")|(
    orders.item_name == "Yerba mate set compleet")|(
    orders.item_name == "Traditioneel Startpakket")

orders["total_quantity"] = np.select([qty2,qty3],[2,3],default=1)

In [27]:
# Rename items

orders["item_name"] = orders["item_name"].str.replace("Traditioneel Startpakket","yerba mate, bombilla, kalebas")
orders["item_name"] = orders["item_name"].str.replace("Yerba mate set 100 gram incl. kalebas en bombilla","yerba mate 100 gram, bombilla, kalebas")
orders["item_name"] = orders["item_name"].str.replace("Yerba Mate Set 100 gram incl. kalebas","yerba mate 100 gram, kalebas")

orders["item_name"] = orders["item_name"].str.replace("Yerba Mate Set 100 gram","yerba mate 100 gram, bombilla, kalebas")
orders["item_name"] = orders["item_name"].str.replace("Yerba Mate Set 250 gram","yerba mate 250 gram, bombilla, kalebas")
orders["item_name"] = orders["item_name"].str.replace("Yerba Mate Set 300 gram","yerba mate 300 gram, bombilla, kalebas")

orders["item_name"] = orders["item_name"].str.replace("Yerba Mate Theezakjes 10 stuks","yerba mate")
orders["item_name"] = orders["item_name"].str.replace("Yerba Mate Theezakjes","yerba mate")
orders["item_name"] = orders["item_name"].str.replace("Gratis Yerba Mate Thee Sample","yerba mate sample")
orders["item_name"] = orders["item_name"].str.replace("Free Yerba Mate Tea Sample","yerba mate sample")
orders["item_name"] = orders["item_name"].str.replace("Yerba Mate Losse Thee","yerba mate")
orders["item_name"] = orders["item_name"].str.replace("Yerba Mate 50 gram","yerba mate 50 gram")
orders["item_name"] = orders["item_name"].str.replace("Yerba Mate 100 gram","yerba mate 100 gram")
orders["item_name"] = orders["item_name"].str.replace("Yerba Mate 250 gram","yerba mate 250 gram")

orders["item_name"] = orders["item_name"].str.replace("Zilveren Bombilla Groot","bombilla")
orders["item_name"] = orders["item_name"].str.replace("Bombilla Zilver","bombilla")
orders["item_name"] = orders["item_name"].str.replace("Bamboo Bombilla","bombilla")
orders["item_name"] = orders["item_name"].str.replace("Bombilla","bombilla")

orders["item_name"] = orders["item_name"].str.replace("Gourd Brown","kalebas")
orders["item_name"] = orders["item_name"].str.replace("Kalebas Bruin","kalebas")
orders["item_name"] = orders["item_name"].str.replace("Keramische Kalebas Eef Gray Red White Dots","kalebas")
orders["item_name"] = orders["item_name"].str.replace("Keramische Kalebas Eef Gray White Marl Large","kalebas")
orders["item_name"] = orders["item_name"].str.replace("Keramische Kalebas Eef Gray Brown Green","kalebas")
orders["item_name"] = orders["item_name"].str.replace("Keramische Kalebas Eef Gray Black Yellow","kalebas")
orders["item_name"] = orders["item_name"].str.replace("Keramische Kalebas Eef Dark Blue Orange","kalebas")

orders["item_name"] = orders["item_name"].str.replace("Yerba Mate","yerba mate")
orders["item_name"] = orders["item_name"].str.replace("Yerba mate","yerba mate")


orders = orders.rename(columns={"item_name":"items"})


In [28]:
products = orders.copy()
products

,order_number,date,item_price,shipping,postal_code,city,country,email,items,quantity,order_total,total_quantity
0,586,2019-06-29 14:28:32,50.37,0.0,1811BD,Alkmaar,NL,a.kashanimaleki@gmail.com,"yerba mate 100 gram, bombilla, kalebas",2,50.37,3
1,586,2019-06-29 14:28:32,50.37,0.0,1811BD,Alkmaar,NL,a.kashanimaleki@gmail.com,yerba mate,1,50.37,1
2,435,2018-06-17 21:17:45,33.02,0.0,1171WS,Badhoevedorp,NL,abdelalaiz@gmail.com,"yerba mate 250 gram, bombilla, kalebas",1,33.02,3
3,397,2018-04-03 13:57:05,24.79,0.0,6814JN,Arnhem,NL,ac_charlotte@hotmail.com,"yerba mate 100 gram, kalebas",1,24.79,2
4,390,2018-03-25 12:06:01,22.00,0.0,5616SV,Eindhoven,NL,adam.klimes689@gmail.com,kalebas,1,22.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...
378,354,2018-02-14 13:10:10,14.88,0.0,1056HR,Amsterdam,NL,zaar10@hotmail.com,Keramische Kalebas Eef Green Blue Dots,1,14.88,1
379,354,2018-02-14 13:10:10,14.88,0.0,1056HR,Amsterdam,NL,zaar10@hotmail.com,bombilla,1,14.88,1
380,388,2018-03-23 9:25:10,33.06,0.0,3523XX,Utrecht,NL,zakariaelbennay@gmail.com,"yerba mate 300 gram, bombilla, kalebas",1,33.06,3
381,421,2018-05-01 22:11:11,0.00,0.0,7041AV,s-Heerenberg,NL,NaN,bombilla,1,0.00,1


In [7]:
# First sort on date
orders = orders.sort_values("date")

# Create variable to join date
combine_text = lambda x: ", ".join(x.unique())
customers = orders.groupby(["email"]).agg({"order_total":"count","date":combine_text}).reset_index()

In [8]:
# Create data frame with split value columns 
data = customers

date_split = data["date"].str.split(",", n = 2, expand = True) 

# Create separate first order date column from new data frame 
data["first_order_date"] = date_split[0]

# Create separate last order date from new data frame 
data["last_order_date"] = date_split[2]
data["last_order_date"] = data["last_order_date"].fillna(date_split[1])
data["last_order_date"] = data["last_order_date"].fillna(date_split[0])
# data["last_order_date"] = data["date"].max()

# Dropping old date column
data.drop(columns = ["date"], inplace = True)


In [9]:
orders["order_number"] = orders["order_number"].astype(str)


# Calculate total revenue from same customer
order_total = orders.groupby(["email"]).agg({"order_total":"sum","quantity":"sum","order_number":combine_text}).reset_index()
order_total.columns=["email","revenue","quantity","order_number"]
# order_total = order_total.sort_values("revenue", ascending=False)

# Calculate total orders from same customer
order_count = orders.groupby(["email"]).agg({"order_total":"count"}).reset_index()
order_count.columns=["email","orders"]
order_count = order_count.sort_values("orders", ascending=False)

In [10]:
# Only include columns we need
orders = orders[orders.columns[orders.columns.isin(["postal_code","city","country","email","item_name"])]]


In [11]:
# Merge total revenue and order count to data
data = pd.merge(customers, orders, on="email")
data = pd.merge(data,order_total, on="email")
data = pd.merge(data,order_count, on="email")

In [13]:
# Drop duplicates (Some customers have duplicate info because they have other adress)
before = len(data)
data = data.drop_duplicates(subset="email", keep="last")
after = len(data)
print("Number of rows dropped",str(before-after))

Number of rows dropped 127


In [14]:
# Drop unnecessary column
data = data.drop("order_total",axis=1)

In [15]:
data.columns

Index(['email', 'first_order_date', 'last_order_date', 'postal_code', 'city',
       'country', 'revenue', 'quantity', 'order_number', 'orders'],
      dtype='object')

In [16]:
# Reorder column names
data = data[["email","postal_code","city","country","first_order_date","last_order_date","order_number","orders","quantity","revenue"]]


In [17]:
# Check NaN values
data.isnull().sum()

email               0
postal_code         1
city                1
country             0
first_order_date    0
last_order_date     0
order_number        0
orders              0
quantity            0
revenue             0
dtype: int64

In [29]:
# Export data to csv
data.to_csv("./data/data_ccv.csv", index=False)
products.to_csv("./data/products_ccv.csv", index=False)